# 截取的数据处理部分
下面是宏常量  

In [ ]:

# 数据处理用
import pandas
import numpy
import time
import datetime
import random
import itertools



trainType = ["普客", "新空调快速", "新空调特快", "新空调直快", "动车", "高速", "城际"]
stationRegion = ["济南", "济南西", "济南东", "大明湖"]


Excelpath = "data.xlsx"  # Excel时刻表输入文件路径，如爬取的时刻表或是是复制来的
TextPath = "train.txt"  # 游戏时刻表文件输出路径
# 速度和编组以及类型映射关系,0为普速1为动车2为高速，后续修改
species = {'新空调普快': ['120', 'LPPPPPP', 0], '新空调快速': ['120', 'LPPPPPP', 0],
           '新空调特快': ['140', 'LPPPPPP', 0], '新空调直快': ['160', 'LPPPPPP', 0],
           '动车': ['200', 'LPPL', 1], '城际': ['200', 'LPPL', 1], '高速': ['300', 'LPPLLPPL', 2]}
species1 = {'K': ['120', 'LPPPPPP', 0], 'T': ['140', 'LPPPPPP', 0], 'Z': ['160', 'LPPPPPP', 0],
            'D': ['200', 'LPPL', 1], 'C': ['200', 'LPPL', 1], 'G': ['300', 'LPPLLPPL', 2]}

# 车站-编号,掉向,用时以及运行车辆种类映射关系
# 图片左(0)右(1)侧线路key值相同则掉向,
# 国铁车辆行走左侧,2为数据为左侧股道编号
# [车站编号,车站所在侧(0为左侧),车辆进场股道,车辆离场行走股道,到达中心车站所用时间]
station = {'武汉站': ['a', -1, 0, 0, 0 ], '武汉动车所': ['c', 1, 0, 0, 5],
           '京广高速咸宁北方向': ['b', 1, 2, 1, 5 ], '武九客专葛店南方向': ['d', 1, 2, 1, 4],
           '京广高速孝感北方向': ['e', 1, 1, 2, 10 ], '沪蓉线红安西方向': ['f', 0, 1, 2, 10],
           '沪蓉线汉口方向': ['h', 0, 1, 2, 5], '丹水池联络线汉口方向': ['i', 0, 1, 2, 7], }

# 线路和车站关系，主要用于从车站-值获取线路-键
route1 = { '武汉动车所': ["武汉"],
           '京广高速咸宁北方向': [ ], '武九客专葛店南方向': ["葛店南","鄂州",""],
           '京广高速孝感北方向': [ ], '沪蓉线红安西方向': [],
           '沪蓉线汉口方向': ["汉口"], '丹水池联络线汉口方向': [], }
# 两个合在一起写太难看


# 车型关系--待完善，后续待交路表部分完成后会使用对应映射而不是上面的简单类型映射
marshalling = {'CHR380BL': [], 'CHR380B': [], 'CHR380B重连': []}


ThisStation = '济南西站'
TotalPlat = 18
# track = {}


# 模拟器信息
desired_caps = {
    'platformName': 'Android',  # 被测手机是安卓
    'platformVersion': '9',  # 手机安卓版本
    'deviceName': 'emulator-5554:5555',  # 设备名和映射连接端口，安卓手机可以随意填写

    'appPackage': 'com.lltskb.lltskb',  # 启动APP Package名称
    'appActivity': '.ui.splash.SplashActivity',  # 启动Activity名称

    'unicodeKeyboard': True,  # 使用自带输入法，输入中文时填True
    'resetKeyboard': True,  # 执行完程序恢复原来输入法
    'noReset': True,       # 不要重置App
    'newCommandTimeout': 6000,
    'automationName': 'UiAutomator2'
}

## 第三部分，处理获取到的数据并生成游戏样式的时刻表
这部分最为重要，按照游戏中时刻表格式分为若干部分  
车辆信息部分，如车次编号，速度等级等信息  
停站信息部分，有四部分按顺序分别为 停站编号，停站股道，停站时刻，停站时间。这四部分会单独拆开分别生成  
例： G7 COMMUTER 300 LPPLLPPL X1 : f#1#11:17:00#0 d#6#11:23:00#3 a#1#11:33:00#0  

内容|完成度|实现原理|近期改良计划|远期优化计划
---|---|---|---|---
车辆信息|较为完善|目前为仅按照车次编号和类型生成|近期暂无|完善交路表爬取以修改速度级和编组信息立折车次信息完善
停站信息-停站编号|完善|依照游戏自身设置直接映射|暂无|暂无
停站信息-停站股道|较为完善|selenium按照web自动化12306查询获取 <br>appnium部分按照车次信息查询--检票口获得|完善车次查询部分以获得<br>停站检票口信息|使用车站大屏来获得无法离线查询的车站信息
停站信息-停站时刻|完善|按照到站时间和出发时间推算|近期暂无|加上线路所映射行车进路部分
停站信息-停站时间|完善|按照到站时间和出发时间计算|近期暂无|立折车次停站时间等按照交路表推算
整体上|较为完善|使用apply(lambda) 来进列映射整处理而不是按照行单个处理|暂无  

### 第三部分第〇小部分--处理用函数定义
#### 内容  
目前按照上面输入为上面自动化爬取获取到csv样式的txt文件，输出为按照车站为单位的数据  
1. 首先去除空行和重复值等基本操作进行数据提纯   
2. 在统一处理如得到部分信息如停站时间和始发终到的处理，  
3. 可以获得中心车站的停站时间信息等  
4. 同时预先定义需要使用的处理函数  

#### 实现原理  
1. 整体上以处理dataframe为主，文件→dataframe→文本
2. 大量使用.apply相应处理函数来处理列，尽可能少使用匿名函数来增加可读性
3. 由文件读出的大表来初步处理groupby 再对想到较小的表来处理
4.


#### 待完善/优化的部分
近期暂无计划，远期计划加入进路映射以使用线路所来优化自动化  

In [ ]:

def dealTime(datafr: pandas.DataFrame) -> pandas.DataFrame:
    # 对于始发车和终到车进行时间处理，默认相同
    for idx, row in datafr.iterrows():
        if row["到时"] == "--:--":
            row["到时"] = row["开时"]
        elif row["开时"] == "--:--":
            row["开时"] = row["到时"]
    # 计算停站时间
    datafr["到时"] = pandas.to_datetime(datafr["到时"])
    datafr["开时"] = pandas.to_datetime(datafr["开时"])
    # 实发终到设置停站时间
    # pd["停站时间"].replace(0,10,inplace=True)

    return datafr


def stopStTime(stopTime: pandas.Timedelta) -> int:
    a = int(stopTime.seconds/60)  # 处理为数值类型的分钟时间
    # 始发或是终到的时间是随机的
    return a if a > 0 else random.randint(10, 30)


def ModtimeStr(switchTime: pandas.Timestamp) -> str:
    # 处理为仅有时分秒样式的字符串格式的时间
    return switchTime.strftime("%H:%M:%S")


def speMarType(traincode: str) -> str:  # 目前获取编组信息的方式
    til = species1[traincode[0]]  # 处理获取车辆信息字符串部分
    return "{cod} COMMUTER {speed} {mar} X1".format(cod=traincode, speed=til[0], mar=til[1])


def checkin(entrance: str) -> int:
    # 终到车
    res = 0
    if entrance == "无检票口信息":
        res = 0
    else:  # 格式类似检票口15A，替换掉其他字样
        res = int(entrance.strip("检票口AB"))
    return res


def prevnextST(stopSt: list[str], stIdx: int) -> list[str]:
    # 返回结果，获取目标车站的前后车站
    res = ["", ""]
    res1 = ["", ""]
    if stIdx == 0:  # 始发车视为始发车站和下一站
        res = [stopSt[0], stopSt[1]]
    elif stIdx == len(stopSt)-1:  # 终到车视为前一站和终到站
        res = [stopSt[len(stopSt)-2], stopSt[len(stopSt)-1]]
    else:  # 中间站
        res = [stopSt[stIdx-1], stopSt[stIdx+1]]
    # 判断并修改进路
    for k, v in route1.items():
        # 把车站名映射为线路
        if res[0] in v:
            res1[0] = k
        elif res[1] in v:
            res1[1] = k
    return res1


def arrLeaTime(t1: pandas.Timestamp, st: list[str], mark: int) -> str:
    # 选取是上一站还是下一站
    tarst = st[mark]
    useTime = station[tarst][4]
    if mark == 0:  # 进场减时间
        res = t1-datetime.timedelta(minutes=useTime)
    else:  # 离场加时间
        res = t1+datetime.timedelta(minutes=useTime)
    # 返回时分秒格式的字符串
    return res.strftime("%H:%M:%S")


### 第三部分第一小部分--爬取信息整体初步处理 
#### 内容


#### 实现原理


#### 待完善/优化的部分

In [ ]:
# 手动建立映射关系,别的方法不会


def processTrains(fn: str, targetSt: str) -> pandas.DataFrame:
    # 处理产生初步所需信息的数据框
    trainsFrame = pandas.read_csv(filepath_or_buffer=fn,
                                  sep=",", encoding="utf8", header=None)
    trainsFrame.columns = ["车次",  "停站", "到时", "开时", "检票口", "drop1"]
    trainsFrame.drop(columns=["drop1"], inplace=True)

    trainsFrame = dealTime(trainsFrame)  # 初步处理时间
    trainsFrame.sort_values(by=["车次", "到时"], inplace=True)  # 归类排序
    # 整理为分钟样式的停站时间
    #trainsFrame["停站时间"] = trainsFrame["停站时间"].apply(lambda x: stopStTime(x))
    trainsFrame.reset_index(drop=True, inplace=True)
    # 按照车次分组为列表，同时车次不作为索引以便于操作
    tfgp = trainsFrame.groupby(by=["车次"], as_index=False).agg(list)
    # 获取目标停站的索引值，设置为一个辅助列
    tfgp["auxIdx"] = tfgp['停站'].apply(lambda x: x.index(targetSt))

    # 获取前后停站以得到更好的交路映射
    tfgp["目标前后站"] = tfgp.apply(
        lambda x: prevnextST(x["停站"], x["auxIdx"]), axis=1)
    # 获取其他信息
    tfgp["目标站到时"] = tfgp.apply(lambda x: x["到时"][x["auxIdx"]], axis=1)
    tfgp["目标站开时"] = tfgp.apply(lambda x: x["开时"][x["auxIdx"]], axis=1)
    tfgp["目标站检票口"] = tfgp.apply(lambda x: x["检票口"][x["auxIdx"]], axis=1)

    # 提取需要的信息为新的dataframe
    trainInfoNeed = tfgp[["车次", "目标站到时", "目标站开时", "目标站检票口", "目标前后站"]]
    trainInfoNeed.columns = ["车次", "到时", "开时", "检票口", "前后站"]
    # print(trainInfoNeed.head())

    return trainInfoNeed


###  第三部分第二小部分 -- 处理生成最终dataframe样式的数据 

#### 内容
按照上面的函数处理数据，最终处理生成有进(离)场时间/股道 等信息的dataframe,之后生成字符串导出    
这里分为两部分，车辆信息的trainframe和路径时刻表的stationframe,  
 'D8176 COMMUTER 200 LPPL X1 : b#2#18:08:00#0 d#0#18:16:00#14 e#0#18:36:00#0 ',

#### 实现原理
> 1. generateTrainInfo 即为文字版时刻表分号前的内容，生成车次信息部分  
>> 1.1. 考虑到内容只需做车次编号，速度等级和编组信息则按照车次首位ktz生成    
>> 2.1. 编组信息尚未准备开始   
> 2. generateArriveLeave 即为文字版时刻表分号后的内容，生成进场和离场信息部分  
>> 2.1. 基于停站的上一站完善进路的刻画
>> 2.2. 本身进场和离场处理基本类似，这里使用一个mark来进行区分
> 3. generateMainSt 即为区域核心车站生成部分  
>> 3.1. 绝大多数信息已经在前面处理完成，这里主要是处理格式问题  
>> 3.2. 车站名称的填充放在最后，防止由于数据全空导致填充失败  

#### 改进计划  
1. 对于分场式车站采用进场线路--对应线路所来映射，加入线路所映射部分


In [ ]:


def generateRoute(dataser: pandas.Series):
    # 进行中，对于分场式车站采用线路所映射进路
    s1 = "黄河南线路所"
    s2 = "大漠刘线路所"
    return None
#


def generateArriveLeave(datafr: pandas.DataFrame, mark) -> pandas.DataFrame:
    # 生成进场/离场信息
    # mark=0为进场，mark=1为离场
    ALStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息
    # 按照信息来映射进场/离场线路
    ALStDF['车站名称'] = datafr["前后站"].apply(lambda x: x[mark])
    # 按照信息来映射进场/离场股道
    ALStDF['股道'] = ALStDF['车站名称'].apply(lambda x: station[x][2+mark])
    # 时间作差
    ALStDF['到达时间'] = datafr.apply(
        lambda x: arrLeaTime(x["到时"], x["前后站"], mark), axis=1)
    # 进程默认经过不停车
    ALStDF['停站时间'] = 0

    return ALStDF


def generateMainSt(datafr: pandas.DataFrame):
    # 生成中心停站信息
    MainStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息

    # 计算停站时长，得到整形类型的时长，非浮点数没有.0便于写入文件
    MainStDF["停站时间"] = (datafr["开时"]-datafr["到时"]
                        ).apply(lambda x: stopStTime(x))
    # 计算并规整进场离场和停站时间，将三个到时改为游戏时分秒格式 hh:mm:ss
    MainStDF["到达时间"] = datafr["到时"].apply(lambda x: ModtimeStr(x))
    # 按照相应格式处理车站检票口为对应形式
    MainStDF["股道"] = datafr["检票口"].apply(lambda x: checkin(x))
    # 填充车站为中心车站
    MainStDF["车站名称"].fillna(value="济南西", inplace=True)
    return MainStDF


def generateTrainInfo(dataser: pandas.Series) -> pandas.Series:
    # 生成车次信息
    res = dataser.apply(lambda x: speMarType(x))
    print(res.head())
    return res


## 第四部分 -- 生成对应样式的字符串并写入文件

### 内容  
1. 整合之前的数据来生成符合样式的dataframe数据合集  
2. 生成csv长字符串并按行分割为一个个的端数据  
3. 最终连接处理生成结果，不替换代号便于后续检查    

### 实现原理
整合数据和字符串处理

### 改进计划  
暂无

In [ ]:

def generateStr(fn: str)->list:
    #生成游戏样式的字符串
    f = open(file=fn, encoding="utf8", mode="w")
    res=[]
    for i in range(0, len(arriveStDF)):
        #format为对应顺序格式
        tf = "{train} : {arrive} {stop} {leave}".format(
            train=trainstr[i], arrive=arrivestr[i], stop=stopstr[i], leave=leavestr[i])
        print(tf)
        res.append(tf) #将结果写入文件
        f.write(tf+"\n")

    f.close()
    return res


at = processTrains("ltInfo.txt", "济南西")
#生成dataframe格式的数据
arriveStDF = generateArriveLeave(at, mark=0)
leaveStDF = generateArriveLeave(at, mark=1)
stopStDF = generateMainSt(at)
trainDF = generateTrainInfo(at["车次"])

#按照四部分生成需求的字符格式，使用to_csv函数来生成
arrivestr = arriveStDF.to_csv(
    sep="#", header=False, index=False).split(sep="\r\n")
leavestr = leaveStDF.to_csv(sep="#", header=False, index=False).split(sep="\r\n")
stopstr = stopStDF.to_csv(sep="#", header=False, index=False).split(sep="\r\n")
trainstr = trainDF.to_csv(header=False, index=False).split(sep="\r\n")

generateStr("测试.txt")



-------  

## 第五部分，可视化用于分析分布如高峰期和主要类型

In [ ]:

# 对于该车站群组（地区）时间段内的所有车次信息


def getStationProid(datafr: pandas.DataFrame, at: int) -> dict:
    # 预先分类，按照小时
    datagruop = datafr.groupby(["到时-小时", "停站", "列车类型"])
    # print(datagruop.groups)
    regionInfo = {}

    # 对于每个车站的每种类型的列车
    for sr in stationRegion:
        # 好像只能分开写两层了
        regionInfo[sr+"trainCode"] = []
        regionInfo[sr+"trainCount"] = []
        for tt in trainType:
            try:  # 查找符合条件的群组
                i = datagruop.get_group((at, sr, tt))

            except KeyError:  # 索引找不到就是0
                regionInfo[sr+"trainCount"].append(0)
                # regionInfo["trainCode"].append(None)
            else:  # 如果没问题，添加回去长度

                regionInfo[sr+"trainCode"].append(i["车次"].tolist())
                regionInfo[sr+"trainCount"].append(len(i))
        regionInfo[sr+"trainCode"] = sum(regionInfo[sr+"trainCode"], [])
    # 解包嵌套的列表
    # print(regionInfo)

    return regionInfo


def getTrainProid(datafr: pandas.DataFrame, at: int) -> dict:
    # 预先分类，数据分析用
    datagruop = datafr.groupby(["到时-小时", "停站", "列车类型"])
    # print(datagruop.groups)
    regionInfo = {}

    # 对于每个车站的每种类型的列车
    for tt in trainType:
        # 好像只能分开写两层了
        regionInfo[tt+"trainCode"] = []
        regionInfo[tt+"stationName"] = []
        regionInfo[tt+"stationCount"] = []
        for st in stationRegion:
            try:  # 查找符合条件的群组
                i = datagruop.get_group((at, st, tt))

            except KeyError:  # 索引找不到就是0
                regionInfo[tt+"stationCount"].append(0)
                # regionInfo["trainCode"].append(None)
            else:  # 如果没问题，添加回去长度
                regionInfo[tt+"trainCode"].append(i["车次"].tolist())
                regionInfo[tt+"stationName"].append(i["停站"].tolist())
                regionInfo[tt+"stationCount"].append(len(i))

        regionInfo[tt+"stationName"] = sum(regionInfo[tt+"stationName"], [])
    # 解包嵌套的列表
    # print(regionInfo)

    return regionInfo
